In [1]:
import os
os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))
    
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data, rel_disp
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib

In [3]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()):
            plot_data = results[coord]['numerical'][opts]
            m_diff, n_diff = [], []
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
#                     deltami, deltani = plotter.mass_difference(mnorm, plot_data[i], results[coord]['grid']['rh'])
#                     m_diff.append(deltami)
#                     n_diff.append(deltani)
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'",""],
                            [": True",""],
                            ["_", " "],
                            ["{",""],["}",""],[","," "]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
                
                dp_dr = 1
                numeric_rel_d = rel_disp(results[coord]['grid']['rh'], plot_data[i]/dp_dr, psi_coord)
                analytic_rel_d = rel_disp(results[coord]['grid']['rh'], results[coord]['analytical'][i]/dp_dr, psi_coord)
                print("analytic rd", analytic_rel_d)
                print("numeric rd", numeric_rel_d)
                dispersion_ratio = numeric_rel_d / analytic_rel_d
                print(f"dispersion ratio of {opts}", dispersion_ratio)
                print("\n")
        print(f"grid_layout: {coord}")
        show_plot(filename)

In [4]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(r0=1, base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}

In [5]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=('n','m'), filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    3.7s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


grid_layout: x_log_of_pn


In [6]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    4.0s finished


analytic rd 0.35718223582650943 dimensionless
numeric rd 0.3569668694267919 dimensionless
dispersion ratio of upwind 0.9993970405632878 dimensionless


analytic rd 0.12575571366996477 dimensionless
numeric rd 0.14969083028424135 dimensionless
dispersion ratio of upwind 1.1903302515310936 dimensionless


analytic rd 0.0673024545137727 dimensionless
numeric rd 0.09911854639733908 dimensionless
dispersion ratio of upwind 1.4727330097159468 dimensionless


grid_layout: x_id


In [7]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   30.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   30.4s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of MPDATA 2 iterations 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.14998377531234405 dimensionless
dispersion ratio of MPDATA 2 iterations 1.0996080858149877 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.10024458785065236 dimensionless
dispersion ratio of MPDATA 2 iterations 1.382726429462366 dimensionless


analytic rd 0.35543054923817785 dimensionless
num

In [8]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   21.8s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   21.8s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of MPDATA 2 iterations 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.14998377531234405 dimensionless
dispersion ratio of MPDATA 2 iterations 1.0996080858149877 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.10024458785065236 dimensionless
dispersion ratio of MPDATA 2 iterations 1.382726429462366 dimensionless


analytic rd 0.35543054923817785 dimensionless
num

In [9]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   32.9s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   32.9s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of MPDATA 2 iterations 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.14998377531234405 dimensionless
dispersion ratio of MPDATA 2 iterations 1.0996080858149877 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.10024458785065236 dimensionless
dispersion ratio of MPDATA 2 iterations 1.382726429462366 dimensionless


analytic rd 0.35543054923817785 dimensionless
num

In [10]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':3},{'n_iters':3,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   47.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   47.2s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of MPDATA 3 iterations 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1472305772422778 dimensionless
dispersion ratio of MPDATA 3 iterations 1.0794229767697567 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.09555085889408232 dimensionless
dispersion ratio of MPDATA 3 iterations 1.3179833523532913 dimensionless


analytic rd 0.35543054923817785 dimensionless
num

In [11]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   54.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   54.2s finished


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of upwind 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.1624758666472522 dimensionless
dispersion ratio of upwind 1.1911940231071916 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.11811558709803255 dimensionless
dispersion ratio of upwind 1.6292305401587903 dimensionless


analytic rd 0.35543054923817785 dimensionless
numeric rd 0.3552171080509709 dimensionless
dispersion ratio of MPDATA 2 iterations 0.999399485531943 dimensionless


analytic rd 0.13639748310979524 dimensionless
numeric rd 0.14998377531234405 dimensionless
dispersion ratio of MPDATA 2 iterations 1.0996080858149877 dimensionless


analytic rd 0.07249777375675796 dimensionless
numeric rd 0.10024458785065236 dimensionless
dispersion ratio of MPDATA 2 iterations 1.382726429462366 dimensionless


analytic rd 0.35543054923817785 dimensionless
num